In [1]:
%%time
# Margin and lots extraction

# STATUS: Complete
# Runtime: 5 secs

import requests
import datetime
import pandas as pd

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'

#******     Streamline pickled option symbols     ****
#_____________________________________________________

df_ib_options = pd.read_pickle(datapath+'df_nse_options.pkl')

# ***** Get list with Mlots + Margin% from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

df_paisa1 = df_paisa[['Symbol', 'Mlot', 'TotMgn%']].sort_values('Symbol').reset_index(drop=True)

# Rename Symbol and Margin fields
df_paisa1 = df_paisa1.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa1 = df_paisa1.apply(pd.to_numeric, errors='ignore')
df_paisa1.marginpct = df_paisa1.marginpct.div(100)

# Dataframe lookup between nseSymbol and ibSymbol
df_ib_nse = pd.concat([pd.read_pickle(datapath+'df_nse_eq_symbols.pkl'), 
            pd.read_pickle(datapath+'df_nse_idx_symbols.pkl')]).\
            reset_index(drop=True)[['ibSymbol','nseSymbol']]

# Merge to get the ibSymbol
df_paisa2 = df_paisa1.merge(df_ib_nse, on='nseSymbol')

df_paisa2.to_pickle(datapath+'df_underlying.pkl')

Wall time: 6.48 s


In [ ]:
df_paisa2